In [1]:
import cv2
import torch
import os
import random
import math
import glob 
import pathlib
import csv, PIL
import time, zipfile
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from tqdm import tqdm
from PIL import Image
from os import listdir
from torchvision import*

%matplotlib inline

## Video and Image merge

In [7]:
total_img  =  glob.glob('/home/Sean/PoCOVID/4real/covid19_pocus_ultrasound_pytorch/code/data/vid2img'+os.sep+"/*")
print(len(total_img))

7710


In [8]:
video_csv = pd.read_csv('/home/Sean/PoCOVID/4real/covid19_pocus_ultrasound_pytorch/code/data/video.csv')
video_csv

,Filename,Filename2
0,Cov_combatting_Image1,Cov_combatting_Image1_frame_003.jpg
1,Cov_combatting_Image1,Cov_combatting_Image1_frame_006.jpg
2,Cov_combatting_Image1,Cov_combatting_Image1_frame_009.jpg
3,Cov_combatting_Image1,Cov_combatting_Image1_frame_012.jpg
4,Cov_combatting_Image1,Cov_combatting_Image1_frame_015.jpg
...,...,...
7705,Reg-Grep-Normal,Reg-Grep-Normal_frame_021.jpg
7706,Reg-Grep-Normal,Reg-Grep-Normal_frame_024.jpg
7707,Reg-Grep-Normal,Reg-Grep-Normal_frame_027.jpg
7708,Reg-Grep-Normal,Reg-Grep-Normal_frame_030.jpg


In [9]:
clean_meta = pd.read_csv('/home/Sean/PoCOVID/4real/covid19_pocus_ultrasound_pytorch/code/data/sean_clean_meta.csv')
clean_meta

,path,Filename,Filename2,Effusion,Consolidations,B-lines,Healthy,Label
0,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov-Atlas+(44),Cov-Atlas+(44).gif,0,1,0,0,COVID-19
1,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov-Atlas-Day+2,Cov-Atlas-Day+2.gif,1,1,1,0,COVID-19
2,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov-Atlas-Day+4,Cov-Atlas-Day+4.gif,1,1,0,0,COVID-19
3,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Atlas-lungcurtain,Reg-Atlas-lungcurtain.gif,0,0,0,1,regular
4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov-Atlas-+(43),Cov-Atlas-+(43).gif,0,0,1,0,COVID-19
...,...,...,...,...,...,...,...,...
176,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_denault_proposedUS_vid11,Cov_denault_proposedUS_vid11.mov,1,1,0,0,COVID-19
177,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_denault_proposedUS_vid12,Cov_denault_proposedUS_vid12.mov,1,1,0,0,COVID-19
178,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_denault_proposedUS_vid13,Cov_denault_proposedUS_vid13.mov,0,0,1,0,COVID-19
179,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_denault_proposedUS_vid14,Cov_denault_proposedUS_vid14.mov,0,0,1,0,COVID-19


In [10]:
result = pd.merge(video_csv,
                 clean_meta[['path', 'Filename', 'Filename2', 'path', 'Effusion','Consolidations','B-lines','Healthy','Label']],
                 on='Filename')
result.head()

,Filename,Filename2_x,path,Filename2_y,path,Effusion,Consolidations,B-lines,Healthy,Label
0,Cov_combatting_Image1,Cov_combatting_Image1_frame_003.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1.mp4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,1,1,COVID-19
1,Cov_combatting_Image1,Cov_combatting_Image1_frame_006.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1.mp4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,1,1,COVID-19
2,Cov_combatting_Image1,Cov_combatting_Image1_frame_009.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1.mp4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,1,1,COVID-19
3,Cov_combatting_Image1,Cov_combatting_Image1_frame_012.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1.mp4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,1,1,COVID-19
4,Cov_combatting_Image1,Cov_combatting_Image1_frame_015.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1.mp4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,1,1,COVID-19


In [16]:
df_new = result.rename(columns={'Filename2_x': 'converted_filename' ,'Filename2_y': 'origin_filename'})
df_new

,Filename,converted_filename,path,origin_filename,path,Effusion,Consolidations,B-lines,Healthy,Label
0,Cov_combatting_Image1,Cov_combatting_Image1_frame_003.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1.mp4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,1,1,COVID-19
1,Cov_combatting_Image1,Cov_combatting_Image1_frame_006.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1.mp4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,1,1,COVID-19
2,Cov_combatting_Image1,Cov_combatting_Image1_frame_009.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1.mp4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,1,1,COVID-19
3,Cov_combatting_Image1,Cov_combatting_Image1_frame_012.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1.mp4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,1,1,COVID-19
4,Cov_combatting_Image1,Cov_combatting_Image1_frame_015.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1.mp4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,1,1,COVID-19
...,...,...,...,...,...,...,...,...,...,...
7247,Reg-Grep-Normal,Reg-Grep-Normal_frame_021.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal.gif,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,0,1,regular
7248,Reg-Grep-Normal,Reg-Grep-Normal_frame_024.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal.gif,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,0,1,regular
7249,Reg-Grep-Normal,Reg-Grep-Normal_frame_027.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal.gif,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,0,1,regular
7250,Reg-Grep-Normal,Reg-Grep-Normal_frame_030.jpg,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal.gif,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,0,0,0,1,regular


In [21]:
df_new = df_new[['path', 'Filename', 'origin_filename', 'converted_filename', 'Effusion', 'Consolidations', 'B-lines' , 'Healthy',  'Label']]
df_new

,path,path,Filename,origin_filename,converted_filename,Effusion,Consolidations,B-lines,Healthy,Label
0,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_003.jpg,0,0,1,1,COVID-19
1,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_006.jpg,0,0,1,1,COVID-19
2,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_009.jpg,0,0,1,1,COVID-19
3,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_012.jpg,0,0,1,1,COVID-19
4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_015.jpg,0,0,1,1,COVID-19
...,...,...,...,...,...,...,...,...,...,...
7247,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_021.jpg,0,0,0,1,regular
7248,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_024.jpg,0,0,0,1,regular
7249,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_027.jpg,0,0,0,1,regular
7250,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_030.jpg,0,0,0,1,regular


In [20]:
df_new = df_new[['path', 'Filename', 'origin_filename', 'converted_filename', 'Effusion', 'Consolidations', 'B-lines' , 'Healthy',  'Label']]
df_new

,path,path,Filename,origin_filename,converted_filename,Effusion,Consolidations,B-lines,Healthy,Label
0,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_003.jpg,0,0,1,1,COVID-19
1,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_006.jpg,0,0,1,1,COVID-19
2,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_009.jpg,0,0,1,1,COVID-19
3,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_012.jpg,0,0,1,1,COVID-19
4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_015.jpg,0,0,1,1,COVID-19
...,...,...,...,...,...,...,...,...,...,...
7247,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_021.jpg,0,0,0,1,regular
7248,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_024.jpg,0,0,0,1,regular
7249,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_027.jpg,0,0,0,1,regular
7250,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_030.jpg,0,0,0,1,regular


In [18]:
df_new.to_csv('final_csv.csv', index=False)

In [22]:
pd.read_csv('/home/Sean/PoCOVID/4real/covid19_pocus_ultrasound_pytorch/code/data/final_csv.csv')

,path,Filename,origin_filename,converted_filename,Effusion,Consolidations,B-lines,Healthy,Label
0,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_003.jpg,0,0,1,1,COVID-19
1,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_006.jpg,0,0,1,1,COVID-19
2,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_009.jpg,0,0,1,1,COVID-19
3,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_012.jpg,0,0,1,1,COVID-19
4,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Cov_combatting_Image1,Cov_combatting_Image1.mp4,Cov_combatting_Image1_frame_015.jpg,0,0,1,1,COVID-19
...,...,...,...,...,...,...,...,...,...
7247,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_021.jpg,0,0,0,1,regular
7248,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_024.jpg,0,0,0,1,regular
7249,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_027.jpg,0,0,0,1,regular
7250,/home/Sean/PoCOVID/4real/covid19_pocus_ultraso...,Reg-Grep-Normal,Reg-Grep-Normal.gif,Reg-Grep-Normal_frame_030.jpg,0,0,0,1,regular
